In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [3]:
url=r'C:/Users/USERPC/Desktop/duk/sem2/pa/spam.csv'
df=pd.read_csv(url)
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.dtypes

Category    object
Message     object
dtype: object

In [5]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [6]:
df=df.drop_duplicates()

In [7]:
df.shape

(5157, 2)

In [8]:
df['Category'].value_counts()

Category
ham     4516
spam     641
Name: count, dtype: int64

In [9]:
#downsampling for equal ham and spam numbers
from sklearn.utils import resample

ham_df = df[df.Category == 'ham']
spam_df = df[df.Category == 'spam']
ham_downsampled = resample(ham_df, 
                           replace=False,      
                           n_samples=len(spam_df), 
                           random_state=42)     

dfd = pd.concat([ham_downsampled, spam_df])

print(dfd.Category.value_counts())


Category
ham     641
spam    641
Name: count, dtype: int64


In [10]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USERPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USERPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USERPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
stop_words = set(stopwords.words('english'))
punctuations = set(string.punctuation)
wl=WordNetLemmatizer()
# Sample tokenization function
def cleaning(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    tokens_lem = [wl.lemmatize(word) for word in tokens]
    return ' '.join(tokens_lem) 


In [12]:
X=df["Message"]
y=df["Category"]

In [13]:
df['Category']=df['Category'].map({'ham':1,'spam':0})

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
pipeline=Pipeline([
    ("Tfidf",TfidfVectorizer(preprocessor=cleaning)),
    ("Mnb",MultinomialNB())
    ])

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=True)

In [16]:
pipeline.fit(x_train,y_train)
predict=pipeline.predict(x_test)

In [17]:
import joblib
joblib.dump(pipeline, "email_spam_classifier.pkl")
from sklearn.metrics import classification_report
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97      1122
        spam       0.99      0.63      0.77       168

    accuracy                           0.95      1290
   macro avg       0.97      0.82      0.87      1290
weighted avg       0.95      0.95      0.95      1290

